In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.widgets import RunDetails
from azureml.core.datastore import Datastore

workspace = Workspace.from_config()

In [ ]:
#datastore = Datastore(workspace, "demostore")
datastore = workspace.get_default_datastore()

#input_data = Dataset.File.from_files(datastore.path('mlops_timeseries/mlops_forecast_data2013.csv')).as_named_input('input_ds').as_mount()
inout = OutputFileDatasetConfig(destination=(datastore, 'inout/'))

In [ ]:
aml_compute = 'demo-cpucluster1'  # <== Compute Cluster Name 
source_directory = 'scripts'

In [ ]:
run_config_1 = RunConfiguration()
run_config_1.target = aml_compute
run_config_1.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='arima-env.yml')

In [ ]:
run_config_2 = RunConfiguration()
run_config_2.target = aml_compute
run_config_2.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='deploy-env.yml')

### [PipelineParameter Class](https://docs.microsoft.com/ja-jp/python/api/azureml-pipeline-core/azureml.pipeline.core.graph.pipelineparameter?view=azure-ml-py)

In [ ]:
# A param named "model_name_arg" was added
pipeline_model_name = PipelineParameter(name="model_name_arg", default_value="arima-model") 

datapath = DataPath(datastore=datastore, path_on_datastore='mlops_timeseries/mlops_forecast_data.csv')
pipeline_input_name = (PipelineParameter(name="input_data_arg", default_value=datapath),
                           DataPathComputeBinding(mode='mount'))

In [ ]:
train = PythonScriptStep(name='train',
                         script_name='train.py', 
                         source_directory=source_directory,
                         arguments =[
                             '--model_name', pipeline_model_name, 
                             '--input', pipeline_input_name, 
                             '--output', inout],
                         inputs=[pipeline_input_name],
                         runconfig=run_config_1,
                         allow_reuse=False)

In [ ]:
register = PythonScriptStep(name='register',
                         script_name='register.py', 
                         source_directory=source_directory,
                         arguments =['--input', inout.as_input(name='inout')],
                         runconfig=run_config_2,
                         allow_reuse=False)

In [ ]:
deploy = PythonScriptStep(name='deploy',
                         script_name='deploy.py', 
                         source_directory=source_directory,
                         arguments =['--input', inout.as_input(name='inout')],
                         runconfig=run_config_2,
                         allow_reuse=False)

In [ ]:
deploy.run_after(register)

In [ ]:
pipeline_deploy = Pipeline(workspace=workspace, steps=[train, register, deploy], description='Pipeline Arima v4')

In [ ]:
pipeline_deploy.validate()

In [ ]:
model_name = "arima-model-v4"
data_path_name = DataPath(datastore=datastore, path_on_datastore='mlops_timeseries/mlops_forecast_data2013.csv')

pipeline_run = Experiment(workspace, 'pipeline_run_v4').submit(pipeline_deploy,                                       \
                                                               pipeline_parameters={"model_name_arg": model_name,     \
                                                                                    "input_data_arg" : data_path_name \
                                                                                   },                                 \
                                                               regenerate_outputs=True)

In [ ]:
RunDetails(pipeline_run).show()

In [ ]:
%%time
pipeline_run.wait_for_completion(show_output=True)

## Publish Pipeline

In [ ]:
from azureml.pipeline.core import PublishedPipeline
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

In [ ]:
published_pipeline_deploy = pipeline_deploy.publish(name="Pipeline_Arima_Published", description="Pipeline Arima Published", continue_on_step_failure=True)
published_pipeline_deploy

In [ ]:
pipeline_id = published_pipeline_deploy.id  # set pipeline id
print(pipeline_id)
published_pipeline = PublishedPipeline.get(workspace, pipeline_id) # retrieve pipeline object with the id
endpoint1 = published_pipeline.endpoint
print(endpoint1)

In [ ]:
auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

response = requests.post(endpoint1, 
                         headers=aad_token, 
                         json={"ExperimentName": "Pipeline_call_arima_v5",
                               "RunSource": "SDK",
                               "DataPathAssignments": {"input_data_arg": {"DataStoreName": "workspaceblobstore","RelativePath": "mlops_timeseries/mlops_forecast_data2013.csv"}},
                               "ParameterAssignments": {"model_name_arg": "arima-model-v5"}})

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception('Received bad response from the endpoint: {}\n'
                    'Response Code: {}\n'
                    'Headers: {}\n'
                    'Content: {}'.format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

In [ ]:
#response.json()